In [11]:
import requests
import random
import time
import json
import os
import pandas as pd
from bs4 import BeautifulSoup

In [21]:
def get_stock_data():
    ticker = input("Enter the stock ticket symbol: ").strip().upper()
    csv_output_name = input("Enter the csv output name: ")
    xlsx_output_name = input("Enter the xlsx output name: ")
    json_output_name = input("Enter the json output name: ")
    url = f'https://finance.yahoo.com/quote/{ticker}'

    user_agent_list = [
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2226.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36',
        'Mozilla/5.0 (X11; CrOS x86_64 6946.55.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.'
    ]

    max_retries = 5

    for attempt in range(max_retries):
        headers = {
            'User-Agent': random.choice(user_agent_list)
        }
        
        try:
            response = requests.get(url, headers=headers)
            print(f"Status code: {response.status_code}")

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                price = soup.find('span', {'data-testid': 'qsp-price'}).text
                price_change = soup.find('span', {'data-testid': 'qsp-price-change'}).text
                price_change_percent = soup.find('span', {'data-testid': 'qsp-price-change-percent'}).text

                print(f"Stock: {ticker}")
                print(f"Price: {price}")
                print(f"Price change: {price_change}")
                print(f"Price change percent: {price_change_percent}")

                stats_labels = [label.text for label in soup.select('div[data-testid="quote-statistics"] ul li span:first-child')]
                stats_values = [value.text for value in soup.select('div[data-testid="quote-statistics"] ul li span:last-child')]

                financial_data = dict(zip(stats_labels, stats_values))

                for label, value in financial_data.items():
                    print(f"{label}: {value}")

                data = {
                    'Stock': ticker,
                    'Price': price,
                    'Price Change': price_change,
                    'Price Change Percent': price_change_percent,
                    **financial_data
                }

                with open((json_output_name), 'w') as json_file:
                    json.dump(data, json_file, indent=4)
                
                df = pd.DataFrame(list(data.items()), columns=['Data', 'Value'])
                df.to_csv(csv_output_name, index=False)
                df.to_excel(xlsx_output_name, index=False)

                return data
            
            elif response.status_code == 429:
                print("Wait...")
                time.sleep(10 * (attempt+1))
            
            else:
                print(response.status_code)
                break
        
        except Exception as e:
            print(e)
    
    print("Failed to get data")
    return {}

In [23]:
data = get_stock_data()

Status code: 200
Stock: AMC
Price: 3.0600 
Price change: +0.0500 
Price change percent: +(1.66%) 
Previous Close: 3.0100 
Open: 2.9700 
Bid: 3.0500 x 3000
Ask: 3.0600 x 21500
Day's Range: 2.9700 - 3.0800 
52 Week Range: 2.3800 - 11.8800 
Volume: 6,202,258 
Avg. Volume: 11,943,698 
Market Cap (intraday): 1.322B 
Beta (5Y Monthly): 2.09
PE Ratio (TTM): -- 
EPS (TTM): -1.0600 
Earnings Date: May 6, 2025 - May 12, 2025
Forward Dividend & Yield: --
Ex-Dividend Date: Mar 6, 2020
1y Target Est: 3.51 
